In [ ]:
# !pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
pd.set_option('display.max_colwidth', 150)
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
from nltk.stem import *
from nltk.stem.porter import *
import math
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
import requests
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertModel
import torch

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# !pip install python-terrier
# !pip install nltk
import pyterrier as pt
if not pt.started():
  pt.init()

In [ ]:
def scrape_documents(topic, max_documents):
    search_url = f"https://en.wikipedia.org/wiki/{topic}"

    response = requests.get(search_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        document_links = soup.find_all('a', class_='mw-redirect')

        results = []

        for idx, link in enumerate(document_links, start=1):
            title = link.text.strip()
            href = 'https://en.wikipedia.org' + link['href']

            document_response = requests.get(href)
            if document_response.status_code == 200:
                document_soup = BeautifulSoup(document_response.content, 'html.parser')
                document_content = document_soup.find('div', class_='mw-parser-output')
                if document_content:
                    text = document_content.get_text(separator='\n')
                else:
                    text = "No content found"

                document_info = {
                    'title': title,
                    'link': href,
                    'text': text
                }
                results.append(document_info)

            if idx == max_documents:
                break

        return results
    else:
        print("Failed to retrieve page:", response.status_code)
        return []


topic = input("Enter the topic you want to search on: ")
if topic.lower() == "auto":
  topic = "science"

documents_per_topic = 20 #changable
all_results = scrape_documents(topic, documents_per_topic)


for result in all_results:
    print("Title:", result['title'])



Enter the topic you want to search on: history
Title: Ancient Greek
Title: umbrella term
Title: Ancient Greek
Title: romanized
Title: Athenian
Title: Boeotic
Title: social sciences
Title: Homo sapiens sapiens
Title: world view
Title: Western thought
Title: early sociologist
Title: French historians
Title: Modern history
Title: Naval history
Title: archeology
Title: great men
Title: Black history
Title: Labor history
Title: Muriel Chamberlain
Title: Charles Harding Firth


In [ ]:
#see what data i got
def view_diff(all_results, num):
  if len(all_results) > num:
      document = all_results[num]
      print(f"Title of document {num} :", document['title'])
      print(f"Link of document {num} :", document['link'])
      print(f"Text of document {num} :", document['text'])
  else:
      print(f"No results or less than {num} documents found.")

In [ ]:
view_diff(all_results,8)

Title of document 8 : world view
Link of document 8 : https://en.wikipedia.org/wiki/World_view
Text of document 8 : Fundamental cognitive orientation of an individual or society


This article is about the concept. For the WorldView satellite class, see 
DigitalGlobe
. For the World View near-space balloon technology, see 
World View Enterprises
. For the Worldview public television network owned by MHz Networks, see 
MHz Networks § MHz Worldview
.


A 
worldview
 or a 
world-view
 or 
Weltanschauung
 is the fundamental 
cognitive
 orientation of an individual or society encompassing the whole of the individual's or society's 
knowledge
, 
culture
, and 
point of view
.
[1]
  A worldview can include 
natural philosophy
; fundamental, existential, and 
normative
 postulates; or themes, values, emotions, and ethics.
[2]






Etymology
[
edit
]


View of the 
world


The term 
worldview
 is a 
calque
 of the German word 
Weltanschauung
 
[ˈvɛltʔanˌʃaʊ.ʊŋ]
 
ⓘ
, composed of 
Welt
 ('world

In [ ]:
#turn the text I got to sth I can work with
def clean_words_and_remove_spaces(text):
    valid_characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"

    cleaned_words = []
    for letter in text:
        if letter in valid_characters:
            cleaned_words.append(letter)
        else:
            cleaned_words.append(' ')

    cleaned_text = ''.join(cleaned_words)
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

In [ ]:
#after little cleaning print what i have
for i in range(len(all_results)):
    document = all_results[i]
    document['text'] = clean_words_and_remove_spaces(document['text'])

In [ ]:
#view data after little cleaning
view_diff(all_results, 8)

Title of document 8 : world view
Link of document 8 : https://en.wikipedia.org/wiki/World_view
Text of document 8 : Fundamental cognitive orientation of an individual or society This article is about the concept For the WorldView satellite class see DigitalGlobe For the World View near space balloon technology see World View Enterprises For the Worldview public television network owned by MHz Networks see MHz Networks MHz Worldview A worldview or a world view or Weltanschauung is the fundamental cognitive orientation of an individual or society encompassing the whole of the individual s or society s knowledge culture and point of view A worldview can include natural philosophy fundamental existential and normative postulates or themes values emotions and ethics Etymology edit View of the world The term worldview is a calque of the German word Weltanschauung v lt an a composed of Welt world and Anschauung perception or view The German word is also used in English It is a concept fundame

In [ ]:
#make a data frame to work
def put_in_data_frame(all_results):
  df = pd.DataFrame(all_results)
  df['docno'] = range(1, len(df) + 1)
  return df

df = put_in_data_frame(all_results)
df

,title,link,text,docno
0,Ancient Greek,https://en.wikipedia.org/wiki/Ancient_Greek_language,,1
1,umbrella term,https://en.wikipedia.org/wiki/Umbrella_term,Semantic relations involving the type of property An example of the relationship between hyponyms and hypernym Hypernymy and hyponymy are the sema...,2
2,Ancient Greek,https://en.wikipedia.org/wiki/Ancient_Greek_language,,3
3,romanized,https://en.wikipedia.org/wiki/Romanization_of_Ancient_Greek,Rendering Greek in Latin characters For the Wikipedia policies on the romanization of Greek names see Wikipedia Naming conventions Greek This arti...,4
4,Athenian,https://en.wikipedia.org/wiki/Athenian,Coordinates N E N E,5
5,Boeotic,https://en.wikipedia.org/wiki/Boeotic,Coordinates N E N E,6
6,social sciences,https://en.wikipedia.org/wiki/Social_sciences,Branch of science that studies society and its relationships For the integrated field of study intended to promote civic competence see Social stu...,7
7,Homo sapiens sapiens,https://en.wikipedia.org/wiki/Homo_sapiens_sapiens,Classification of the human species Homo humans Temporal range Piacenzian Present Ma Pre O S D C P T J K Pg N Scientific classification Domain Euk...,8
8,world view,https://en.wikipedia.org/wiki/World_view,Fundamental cognitive orientation of an individual or society This article is about the concept For the WorldView satellite class see DigitalGlobe...,9
9,Western thought,https://en.wikipedia.org/wiki/Western_thought,Philosophy of the Western world History of Western philosophy redirects here For the book see A History of Western Philosophy Part of a series on ...,10


In [ ]:
#preproccess
stemmer = PorterStemmer()
def Preprocess_my_text(text):
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"RT ", " ", text)
    text = re.sub(r"@[\w]*", " ", text)
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r"\s+", " ", text)
    tokens = word_tokenize(text)
    filtered_stemmed_tokens = [stemmer.stem(word.lower()) for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_stemmed_tokens)



df['cleaned_text'] = df['text'].apply(Preprocess_my_text).apply(clean_words_and_remove_spaces)
# df['cleaned_text'] = df['text']
df.drop(columns = {"text"},inplace = True)

In [ ]:
df.dropna(inplace = True)
df

,title,link,docno,cleaned_text
0,Ancient Greek,https://en.wikipedia.org/wiki/Ancient_Greek_language,1,
1,umbrella term,https://en.wikipedia.org/wiki/Umbrella_term,2,semant relat involv type properti exampl relationship hyponym hypernym hypernymi hyponymi semant relat gener term hypernym specif instanc hyponym ...
2,Ancient Greek,https://en.wikipedia.org/wiki/Ancient_Greek_language,3,
3,romanized,https://en.wikipedia.org/wiki/Romanization_of_Ancient_Greek,4,render greek latin charact wikipedia polici roman greek name see wikipedia name convent greek articl contain non classic letter greek alphabet may...
4,Athenian,https://en.wikipedia.org/wiki/Athenian,5,coordin n e n e
5,Boeotic,https://en.wikipedia.org/wiki/Boeotic,6,coordin n e n e
6,social sciences,https://en.wikipedia.org/wiki/Social_sciences,7,branch scienc studi societi relationship integr field studi intend promot civic compet see social studi social polit econom theori first pioneer k...
7,Homo sapiens sapiens,https://en.wikipedia.org/wiki/Homo_sapiens_sapiens,8,classif human speci homo human tempor rang piacenzian present pre c p j k pg n scientif classif domain eukaryota kingdom animalia phylum chordata ...
8,world view,https://en.wikipedia.org/wiki/World_view,9,fundament cognit orient individu societi articl concept worldview satellit class see digitalglob world view near space balloon technolog see world...
9,Western thought,https://en.wikipedia.org/wiki/Western_thought,10,philosophi western world histori western philosophi redirect book see histori western philosophi part seri philosophi philosophi portal content ou...


In [ ]:
def update_token_dict(token_dict, tokens, doc_number):
    for token in tokens:
        if token in token_dict:
            doc_found = False
            for idx, (doc, freq) in enumerate(token_dict[token]):
                if doc == doc_number:
                    token_dict[token][idx] = (doc, freq + 1)
                    doc_found = True
                    break
            if not doc_found:
                token_dict[token].append((doc_number, 1))
        else:
            token_dict[token] = [(doc_number, 1)]

tokens_dict = {}

for i in range(len(df)):
    doc = df["cleaned_text"][i]
    words = doc.split()
    update_token_dict(tokens_dict, words, df["docno"][i])

print(tokens_dict)


{'semant': [(2, 24), (7, 1), (10, 1)], 'relat': [(2, 16), (7, 28), (8, 1), (9, 4), (10, 4), (11, 9), (12, 4), (13, 2), (14, 9), (15, 7), (16, 4), (18, 8)], 'involv': [(2, 1), (4, 1), (7, 4), (8, 1), (9, 1), (11, 2), (13, 1), (14, 11), (15, 16), (16, 1)], 'type': [(2, 5), (4, 5), (7, 4), (8, 10), (9, 5), (10, 1), (11, 5), (13, 4), (14, 10), (15, 10), (16, 2), (18, 2)], 'properti': [(2, 1), (7, 1), (10, 1), (13, 1), (15, 22)], 'exampl': [(2, 13), (4, 6), (7, 6), (8, 2), (9, 4), (10, 4), (11, 5), (13, 4), (14, 4), (15, 8), (16, 5), (18, 3)], 'relationship': [(2, 8), (7, 12), (10, 1), (11, 4), (13, 1), (15, 3), (18, 1)], 'hyponym': [(2, 54)], 'hypernym': [(2, 37)], 'hypernymi': [(2, 5)], 'hyponymi': [(2, 12)], 'gener': [(2, 7), (4, 1), (7, 22), (8, 1), (9, 2), (10, 8), (11, 11), (12, 1), (13, 8), (14, 10), (15, 10), (16, 7), (18, 5), (19, 1), (20, 1)], 'term': [(2, 15), (4, 1), (7, 15), (9, 4), (10, 17), (11, 19), (12, 1), (13, 9), (14, 3), (15, 7), (17, 1), (19, 1), (20, 1)], 'specif': [(

In [ ]:
def where_do_i_find_this_word(word, word_dict):
  word = clean_words_and_remove_spaces(word)
  word = Preprocess_my_text(word)
  doc_indices = []
  # print(word)
  # print(word[0])

  for word_of_i, doc_and_freq in word_dict.items():

    if word_of_i == word:
        # print(doc_and_freq)
        # print(word_of_i)
        for doc_num, freq in doc_and_freq:
                doc_indices.append(doc_num)

                # print(doc_num)
        # print(word_of_i)
  return doc_indices


# where_do_i_find_this_word("type",tokens_dict)

def find_query(tokens_dict):
   query = input("Enter what you want: ")
   query = Preprocess_my_text(query)
   query = clean_words_and_remove_spaces(query)
   query = query.split()
   list_of_docs_that_will_get_merged = []
   for i in range(len(query)):

      for word_j , doc_num_and_freq in tokens_dict.items():
        if(word_j == query[i]):
          list_of_docs_that_will_get_merged.append(doc_num_and_freq)
   return list_of_docs_that_will_get_merged,query


def preform_TF_IDF(list_to_merge):
  total_documents = len(list_to_merge)

  idf = {}
  for doc in list_to_merge:
      for term_freq in doc:
          word = term_freq[0]
          if word not in idf:
              doc_count = sum(1 for doc_ in list_to_merge if any(word == terms[0] for terms in doc_))
              idf[word] = math.log(total_documents / doc_count)

  tf_idf = []
  for doc in list_to_merge:
      tf_idf_docs = []
      for term_freq in doc:
          word = term_freq[0]
          tf_idf = term_freq[1] * idf[word]
          tf_idf_docs.append((word, tf_idf))
      tf_idf_docs.append(tf_idf)
  return tf_idf

list_to_merge, query = find_query(tokens_dict) #find based on the inverted index
print()
print(f"After TF_IDF rank of query ({query}) is: ",preform_TF_IDF(list_to_merge))#find based on tf idf


Enter what you want: when di hitler ?? die >? 

After TF_IDF rank of query (['di', 'hitler', 'die']) is:  1.0986122886681098


In [ ]:
#kl 7aga i guess
def find_expand_and_rank(df,query):
  # indexing
  indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
  index_ref = indexer.index(df["cleaned_text"].astype(str), df["docno"].astype(str))
  index_ref.toString()
  index = pt.IndexFactory.of(index_ref)

  #expand
  print(query)
  QE = pt.BatchRetrieve(index, wmodel="TF_IDF", controls={"qemodel" : "Bo1", "qe" : "on"}).search(' '.join(query))

  return QE


q_results = find_expand_and_rank(df,query)

19:07:53.840 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1) - further warnings are suppressed
19:07:55.625 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents
['di', 'hitler', 'die']


In [ ]:
q_results


,qid,docid,docno,rank,score,query
0,1,13,14,0,17.860578,di hitler die
1,1,17,18,1,14.556854,di hitler die
2,1,15,16,2,13.495487,di hitler die
3,1,12,13,3,11.838339,di hitler die
4,1,10,11,4,6.828389,di hitler die
5,1,14,15,5,5.669966,di hitler die
6,1,11,12,6,5.438933,di hitler die
7,1,9,10,7,4.178155,di hitler die
8,1,19,20,8,3.289989,di hitler die
9,1,6,7,9,2.795180,di hitler die


In [ ]:
#RAM crash


# def find_expand_and_rank_elmo_version(df,query):
#   elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=False)
#   tensors = tf.constant(df['cleaned_text'])
#   embeddings = elmo(tensors)
#   with tf.Session() as sess:
#       sess.run(tf.global_variables_initializer())
#       sess.run(tf.tables_initializer())
#       embeddings = sess.run(embeddings)

#   query_embedding = elmo(tf.constant([query]))  # Embedding for the query
#   with tf.Session() as sess:
#       sess.run(tf.global_variables_initializer())
#       sess.run(tf.tables_initializer())
#       query_embedding = sess.run(query_embedding)

#   similarities = []
#   for doc_embedding in embeddings:
#       sim = np.dot(query_embedding, doc_embedding.T) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
#       similarities.append(sim[0][0])

#   # Rank documents based on similarity scores
#   ranked_docs = sorted(zip(df['docno'], similarities), key=lambda x: x[1], reverse=True)

#   return ranked_docs


# ranked_docs = find_expand_and_rank_elmo_version(df, query)

In [ ]:
# print(query)
# df['cleaned_text'][:6].tolist()
# def find_expand_and_rank_elmo_version(df,query):
#   elmo = hub.KerasLayer("https://tfhub.dev/google/elmo/3", trainable=False)
#   tensors = tf.constant(df['cleaned_text'][12:18]) #more than this colab ram will crash

#   tensors_q = tf.constant(query)
#   query_embedding = elmo(tensors_q)
#   embeddings = elmo(tensors)


#   similarities = cosine_similarity(query_embedding, embeddings)



#   ranked_indices = np.argsort(-similarities[0])

#   return df.iloc[ranked_indices]

# find_expand_and_rank_elmo_version(df,query)

In [ ]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# inputs = tokenizer(' '.join(query), return_tensors="pt")
# outputs = model(**inputs)
# word_vect = outputs.pooler_output.detach().numpy()
# word_vect

In [ ]:
def return_the_links(df, q_results):
    df["docno"] = df["docno"].astype(str)
    q_results["docno"] = q_results["docno"].astype(str)

    merged_df = pd.merge(df, q_results, on="docno", how="inner")
    links_list = [(title, link) for title, link in zip(merged_df["title"], merged_df["link"])]
    return links_list


links_list = return_the_links(df, q_results)
print(links_list)


[('social sciences', 'https://en.wikipedia.org/wiki/Social_sciences'), ('Homo sapiens sapiens', 'https://en.wikipedia.org/wiki/Homo_sapiens_sapiens'), ('world view', 'https://en.wikipedia.org/wiki/World_view'), ('Western thought', 'https://en.wikipedia.org/wiki/Western_thought'), ('early sociologist', 'https://en.wikipedia.org/wiki/Sociology_in_medieval_Islam'), ('French historians', 'https://en.wikipedia.org/wiki/French_historians'), ('Modern history', 'https://en.wikipedia.org/wiki/Modern_history'), ('Naval history', 'https://en.wikipedia.org/wiki/Naval_history'), ('archeology', 'https://en.wikipedia.org/wiki/Archeology'), ('great men', 'https://en.wikipedia.org/wiki/Great_Man_theory'), ('Black history', 'https://en.wikipedia.org/wiki/Black_history_(disambiguation)'), ('Labor history', 'https://en.wikipedia.org/wiki/Labor_history_(discipline)'), ('Muriel Chamberlain', 'https://en.wikipedia.org/wiki/Muriel_Chamberlain'), ('Charles Harding Firth', 'https://en.wikipedia.org/wiki/Charles

evaluation


In [ ]:
#mean avg percision #all of topic science
q = "where do we/ study plants?" #let relevent ranked = {10,1,8,3,6}
q2 = "expermints IN science" #let relevent ranked = {6,4,8,6}

# q =  Preprocess_my_text(q)
# q = clean_words_and_remove_spaces(q)
# q = q.split()

# q2 =  Preprocess_my_text(q)
# q2 = clean_words_and_remove_spaces(q)
# q2 = q.split()

import time
start_time = time.time()


def search(query,topic):

    if topic.lower() == "auto":
      topic = "science"

    documents_per_topic = 10 #changable
    all_results = scrape_documents(topic, documents_per_topic)
    # for result in all_results:
    #     print("Title:", result['title'])

    for i in range(len(all_results)):
      document = all_results[i]
      document['text'] = clean_words_and_remove_spaces(document['text'])
    # print(df)
    df = put_in_data_frame(all_results)
    df['cleaned_text'] = df['text'].apply(Preprocess_my_text).apply(clean_words_and_remove_spaces)
    df.drop(columns = {"text"},inplace = True)
    df.dropna(inplace = True)

    query = Preprocess_my_text(query).split()
    q_results = find_expand_and_rank(df,query)

    # links_list = return_the_links(df, q_results)

    return q_results



s1 = search(q, "auto")

s2 = search(q2, "auto")


end_time = time.time()
runtime = end_time - start_time
print("Runtime:", runtime, "seconds")

19:13:10.478 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (3) - further warnings are suppressed
19:13:11.191 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents
['studi', 'plant']
19:13:17.910 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (3) - further warnings are suppressed
19:13:18.677 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 1 empty documents
['expermint', 'scienc']
Runtime: 15.082780838012695 seconds


In [ ]:
s1


,qid,docid,docno,rank,score,query
0,1,9,10,0,9.791346,studi plant
1,1,0,1,1,9.565536,studi plant
2,1,7,8,2,7.154443,studi plant
3,1,5,6,3,3.046027,studi plant
4,1,3,4,4,2.978659,studi plant
5,1,1,2,5,2.667803,studi plant
6,1,8,9,6,2.600743,studi plant
7,1,6,7,7,0.450235,studi plant


In [ ]:
s2

,qid,docid,docno,rank,score,query
0,1,5,6,0,9.182117,expermint scienc
1,1,0,1,1,7.619968,expermint scienc
2,1,3,4,2,4.058577,expermint scienc
3,1,1,2,3,3.549649,expermint scienc
4,1,7,8,4,3.002277,expermint scienc
5,1,9,10,5,2.719211,expermint scienc
6,1,8,9,6,1.769023,expermint scienc
7,1,4,5,7,1.386914,expermint scienc
8,1,6,7,8,0.772536,expermint scienc


In [ ]:
list1 = [10,1,8,3,6]
list2 = [6,4,8,6]

docno1 = s1["docno"].tolist()
docno2 = s2["docno"].tolist()

ap1 = 0
count = 1
for i in range(1,len(docno1)):
  if int(docno1[i]) in list1:
    ap1 += count/i
    count+=1
ap1 = ap1/len(list1)


ap2 = 0
count = 1
for i in range(1,len(docno2)):
  if int(docno2[i]) in list2:
    ap2 += count/i
    count+=1
ap2 = ap2/len(list2)


map = (ap1+ap2)/2
print("mean avg percion", map) #this is with just random predicted relevance the docs i chose in list1 and list2 can be not the best ranking

mean avg percion 0.425
